In [1]:
%%capture
!wget https://www.dropbox.com/scl/fi/uznju0fkgnwz1yi4cmgol/review-Vermont.json.gz?rlkey=4l3mithogu5c08x5lxe6pbb6t&st=wcprbsjo&dl=0
!wget https://www.dropbox.com/scl/fi/16vy3q077mz01n4r2nml1/meta-Vermont.json.gz?rlkey=zu5joaocmaqdg0p17oj14z5lz&st=kih98lc6&dl=0
!mv review-Vermont.json.gz?rlkey=4l3mithogu5c08x5lxe6pbb6t review-Vermont.json.gz
!mv meta-Vermont.json.gz?rlkey=zu5joaocmaqdg0p17oj14z5lz meta-Vermont.json.gz

In [14]:
print("Importando librerias")
import random
from io import BytesIO
from requests import get
import concurrent.futures
import gzip
import json
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
from itertools import chain
print("* Librerias importadas * ")

Importando librerias
* Librerias importadas * 


# Data Creation

In [3]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)


metadata = list(parse("meta-Vermont.json.gz"))
reviews = list(parse("review-Vermont.json.gz"))

In [85]:
len(reviews)

852203

In [16]:
categories = list(business['category'] for business in metadata if business['category'] is not None)
categories = list(chain.from_iterable(categories))
categories = list(set(categories))

In [22]:
restaurant_categories = []
for categ in categories:
    categ_low = categ.lower()
    if 'restaurant' in categ_low or 'restaurante' in categ_low:
        restaurant_categories.append(categ)

In [31]:
gmap_id_to_metadata = {business['gmap_id'] : business for business in metadata}

In [37]:
def verify_is_restaurant(categories):
    # si no presenta informacion, no lo agregamos
    if categories is None:
        return False
    # si es restaurante, retornamos true
    for categ in categories:
        categ_low = categ.lower()
        if 'restaurant' in categ_low:
            return True
    # en otro caso, no se agrega
    return False
restaurant_reviews = []
for rev in reviews:
    gmap_id = rev['gmap_id']
    category_list = gmap_id_to_metadata[gmap_id]['category']
    is_restaurant = verify_is_restaurant(category_list) # booleano
    if is_restaurant:
        restaurant_reviews.append(rev)

In [88]:
print(len(restaurant_reviews) / len(reviews))
print(len(restaurant_reviews))
print(len(reviews))

0.33739965712394815
287533
852203


In [53]:
data = {
    'user_id' : [],
    'gmap_id' : [],
    'rating' : [],
    'text' : [],
    'img_url' : []
}

for review in restaurant_reviews:
    gmap_id = review['gmap_id']
    has_img = review['pics'] is not None
    has_text = review['text'] is not None
    has_all_info = has_img and has_text
    is_duplicated = gmap_id in data['gmap_id']

    if not has_all_info: continue

    if is_duplicated: continue

    # business pic
    pics = review['pics']
    first_pic = pics[0]
    url_first_pic = first_pic['url'][0] # select the first pic

    # business text
    text = review['text']

    # reviews's rating
    rating = review['rating']

    # user id
    user_id = review['user_id']

    data['user_id'].append(user_id)
    data['gmap_id'].append(gmap_id)
    data['rating'].append(rating)
    data['text'].append(text)
    data['img_url'].append(url_first_pic)

df = pd.DataFrame(data)

In [56]:
df.head()

,user_id,gmap_id,rating,text,img_url
0,103949805645203359878,0x4cca9d0401cd8345:0xfd0f1a365e865e14,5,Consistently hits the spot for a straightforwa...,https://lh5.googleusercontent.com/p/AF1QipM3-r...
1,101675045093430739016,0x89e0e7a4069e3d61:0x1af55786531d8d9d,5,From start to finish a perfect experience. Pa...,https://lh5.googleusercontent.com/p/AF1QipNw2B...
2,115300326029580828414,0x89e051f190635a89:0x56ebe0d777708856,5,Cozy cafe with the most delicious sandwiches. ...,https://lh5.googleusercontent.com/p/AF1QipN_df...
3,118345070711245440943,0x89e11b846c8dda3f:0x6ba26f2491aacf4b,5,Great Italian food. We both had veal and sauce...,https://lh5.googleusercontent.com/p/AF1QipM_F9...
4,105352358565599354309,0x89e0e67384a93239:0x99b3594193801be3,5,We visited during week two of when they opened...,https://lh5.googleusercontent.com/p/AF1QipNs-I...


In [67]:
image_data = df[['gmap_id', 'img_url']]
os.makedirs('new_images', exist_ok=True)
indexes_to_drop = []

"""
fuente: https://medium.com/analytics-vidhya/how-to-download-images-faster-using-multithreading-in-python-a31110468770
"""

def download_image(row):
    gmap_id, img_url = row.gmap_id, row.img_url
    specific_data = df[(df.gmap_id == gmap_id) & (df.img_url == img_url)]
    index = specific_data.index[0]
    prefix = 'new_images'
    img_path = f'{prefix}/{gmap_id}.png'

    if os.path.isfile(img_path): # caso en que exista
        print(f"- [  !  ] La imagen {img_path} ya esta en la carpeta")
        return
    response = get(img_url)

    if response.status_code == 200:
        curr_images = len(os.listdir('new_images'))
        print(f"- [ DONE ] ({curr_images}/{image_data.shape[0]}) Descargando y guardando imagen {gmap_id}.png con exito")
        img_bytes = response.content
        with open(img_path, 'wb') as img_file:
            img_file.write(img_bytes)

        df.at[index, 'img_url'] = img_path
    else:
        print(f"- [ FAIL ] Imagen {img_url} no puede ser descargada")
        indexes_to_drop.append(index)

def create_folder():
    print("Creando carpeta 'new_images/'")
    os.makedirs('new_images', exist_ok=True)

    t1 = time.perf_counter()

    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor: # Pueden alterar la cantidad de workers segun el rendimiento
        for row in image_data.itertuples():
            executor.submit(download_image, row)

    t2 = time.perf_counter()

    print(f'Finished in {t2-t1} seconds')


In [68]:
create_folder()

Creando carpeta 'new_images/'
- [  !  ] La imagen new_images/0x4cca9d0401cd8345:0xfd0f1a365e865e14.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x89e0e7a4069e3d61:0x1af55786531d8d9d.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x89e051f190635a89:0x56ebe0d777708856.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x89e11b846c8dda3f:0x6ba26f2491aacf4b.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x89e0e67384a93239:0x99b3594193801be3.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x89e0e5e1565bc49d:0xac5330517455f426.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x4cca7b2aa43ccf0f:0xe6630b7ca2621c25.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x4cb5731feb65652d:0xe49d87a5167bf9a2.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x89e05f768a70138d:0x9254d3d23069a44.png ya esta en la carpeta
- [  !  ] La imagen new_images/0x4cb5eab56a8847af:0x2337c93c5ba4f1a4.png ya esta en la carpeta
- [ FAIL ] Imagen htt

In [69]:
# ojo, probablemente no todos los request sean efectivos,
# por lo tanto, es probable que algunas imagenes no se descarguen
len(os.listdir('new_images'))

1169

In [71]:
%%capture
!zip -r new_images.zip new_images

In [72]:
def image_in_folder(row):
    img_url = row.gmap_id + '.png'
    if img_url not in os.listdir('new_images'):
        return False
    return True

df['image_in_folder'] = df.apply(image_in_folder, axis=1)

In [73]:
df_filtered = df[df.image_in_folder]

In [74]:
df_filtered.drop(columns=['image_in_folder'], inplace=True)

<ipython-input-74-c04a6fb3ae26>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['image_in_folder'], inplace=True)


In [76]:
for row in df_filtered.itertuples():
    filename = row.gmap_id + '.png'
    if filename not in os.listdir('new_images'):
        print(filename)

In [77]:
df_filtered['img_url'] = df_filtered['gmap_id'] + '.png'

<ipython-input-77-d545b7ebf3e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['img_url'] = df_filtered['gmap_id'] + '.png'


In [78]:
df_filtered

,user_id,gmap_id,rating,text,img_url
0,103949805645203359878,0x4cca9d0401cd8345:0xfd0f1a365e865e14,5,Consistently hits the spot for a straightforwa...,0x4cca9d0401cd8345:0xfd0f1a365e865e14.png
1,101675045093430739016,0x89e0e7a4069e3d61:0x1af55786531d8d9d,5,From start to finish a perfect experience. Pa...,0x89e0e7a4069e3d61:0x1af55786531d8d9d.png
2,115300326029580828414,0x89e051f190635a89:0x56ebe0d777708856,5,Cozy cafe with the most delicious sandwiches. ...,0x89e051f190635a89:0x56ebe0d777708856.png
3,118345070711245440943,0x89e11b846c8dda3f:0x6ba26f2491aacf4b,5,Great Italian food. We both had veal and sauce...,0x89e11b846c8dda3f:0x6ba26f2491aacf4b.png
4,105352358565599354309,0x89e0e67384a93239:0x99b3594193801be3,5,We visited during week two of when they opened...,0x89e0e67384a93239:0x99b3594193801be3.png
...,...,...,...,...,...
1179,107484719045053726224,0x4cca7760f3d3bd53:0x53de5ceb2c446bad,5,Really good and drinks and really Amazing Work...,0x4cca7760f3d3bd53:0x53de5ceb2c446bad.png
1180,115446772431703469921,0x89e1dcd157b612e9:0x98cf856f12f834c6,5,All aboard?? It’s a train I would wanna take a...,0x89e1dcd157b612e9:0x98cf856f12f834c6.png
1181,107334404833427814802,0x4cca7a2e25dfcdbd:0x2be66d847fc357f3,2,"I ordered a filet-o-fish, with extra sauce, an...",0x4cca7a2e25dfcdbd:0x2be66d847fc357f3.png
1182,110866039828814938563,0x4cca7b8cb7f7295f:0x6c7b4c03f22025a7,5,"A franchise of the famous Chicago eatery, this...",0x4cca7b8cb7f7295f:0x6c7b4c03f22025a7.png


In [92]:
(len(df_filtered) / len(reviews)) * 100

0.1371738893197982

In [79]:
df_filtered.to_csv('reviews_vermont.csv', index=False)

In [81]:
restaurant_reviews_vermont = restaurant_reviews
for review in restaurant_reviews_vermont:
    gmap_id = review['gmap_id']
    rev_metadata = gmap_id_to_metadata[gmap_id]
    review['metadata'] = rev_metadata


In [84]:
with gzip.open('restaurant_reviews_vermont.json.gz', 'wt') as f:
    json.dump(restaurant_reviews_vermont, f)
